# Generativne mreže

Rekurentne neuronske mreže (RNN-ovi) i njihove varijante s kontroliranim ćelijama, poput ćelija dugog kratkoročnog pamćenja (LSTM-ova) i kontroliranih rekurentnih jedinica (GRU-ova), omogućile su modeliranje jezika, tj. mogu naučiti redoslijed riječi i pružiti predviđanja za sljedeću riječ u nizu. To nam omogućuje korištenje RNN-ova za **generativne zadatke**, poput običnog generiranja teksta, strojnog prevođenja, pa čak i opisivanja slika.

U RNN arhitekturi koju smo raspravili u prethodnoj jedinici, svaka RNN jedinica proizvodila je sljedeće skriveno stanje kao izlaz. Međutim, možemo dodati i drugi izlaz svakoj rekurentnoj jedinici, što bi nam omogućilo da dobijemo **niz** (koji je jednake duljine kao i izvorni niz). Štoviše, možemo koristiti RNN jedinice koje ne primaju ulaz na svakom koraku, već samo uzimaju neki početni vektor stanja i zatim proizvode niz izlaza.

U ovom ćemo se bilježniku usredotočiti na jednostavne generativne modele koji nam pomažu generirati tekst. Radi jednostavnosti, izgradimo **mrežu na razini znakova**, koja generira tekst slovo po slovo. Tijekom treniranja, trebamo uzeti neki korpus teksta i podijeliti ga na nizove slova.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Izgradnja vokabulara znakova

Za izgradnju generativne mreže na razini znakova, potrebno je tekst podijeliti na pojedinačne znakove umjesto na riječi. `TextVectorization` sloj koji smo koristili ranije ne može to učiniti, pa imamo dvije opcije:

* Ručno učitati tekst i izvršiti tokenizaciju 'ručno', kao u [ovom službenom Keras primjeru](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Koristiti `Tokenizer` klasu za tokenizaciju na razini znakova.

Odlučit ćemo se za drugu opciju. `Tokenizer` se također može koristiti za tokenizaciju na razini riječi, pa bi trebalo biti jednostavno prebaciti se s tokenizacije na razini znakova na tokenizaciju na razini riječi.

Za tokenizaciju na razini znakova, potrebno je proslijediti parametar `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Također želimo koristiti jedan poseban token za označavanje **kraja niza**, koji ćemo nazvati `<eos>`. Dodajmo ga ručno u vokabular:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Sada, za kodiranje teksta u nizove brojeva, možemo koristiti:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Treniranje generativne RNN za generiranje naslova

Način na koji ćemo trenirati RNN za generiranje naslova vijesti je sljedeći. U svakom koraku uzet ćemo jedan naslov, koji će se proslijediti u RNN, i za svaki ulazni znak tražit ćemo od mreže da generira sljedeći izlazni znak:

![Slika koja prikazuje primjer generiranja riječi 'HELLO' pomoću RNN-a.](../../../../../translated_images/hr/rnn-generate.56c54afb52f9781d.webp)

Za posljednji znak našeg niza tražit ćemo od mreže da generira `<eos>` token.

Glavna razlika generativne RNN koju ovdje koristimo je ta što ćemo uzimati izlaz iz svakog koraka RNN-a, a ne samo iz završne ćelije. To se može postići postavljanjem parametra `return_sequences` za RNN ćeliju.

Dakle, tijekom treniranja, ulaz u mrežu bit će niz kodiranih znakova određene duljine, a izlaz će biti niz iste duljine, ali pomaknut za jedan element i završen s `<eos>`. Minibatch će se sastojati od nekoliko takvih nizova, a za poravnanje svih nizova trebat ćemo koristiti **padding**.

Napravimo funkcije koje će transformirati skup podataka za nas. Budući da želimo dodati padding na razini minibatcha, prvo ćemo grupirati skup podataka pozivom `.batch()`, a zatim ga `map`-irati kako bismo izvršili transformaciju. Dakle, funkcija transformacije uzimat će cijeli minibatch kao parametar:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Nekoliko važnih stvari koje ovdje radimo:
* Prvo izdvajamo stvarni tekst iz string tenzora
* `text_to_sequences` pretvara popis stringova u popis tenzora s cijelim brojevima
* `pad_sequences` dopunjava te tenzore do njihove maksimalne duljine
* Na kraju vršimo one-hot enkodiranje svih znakova, kao i pomicanje i dodavanje `<eos>`. Uskoro ćemo vidjeti zašto su nam potrebni one-hot enkodirani znakovi

Međutim, ova funkcija je **Pythonic**, tj. ne može se automatski prevesti u Tensorflow računalni graf. Dobit ćemo pogreške ako pokušamo koristiti ovu funkciju izravno u funkciji `Dataset.map`. Moramo ovu Pythonic funkciju obuhvatiti koristeći `py_function` omotač:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Napomena**: Razlikovanje između Pythonovih i Tensorflow funkcija za transformaciju podataka može se činiti prilično složenim, i možda se pitate zašto ne transformiramo skup podataka koristeći standardne Python funkcije prije nego ga proslijedimo u `fit`. Iako se to definitivno može učiniti, korištenje `Dataset.map` ima veliku prednost, jer se pipeline za transformaciju podataka izvršava koristeći Tensorflow-ov računalni graf, što omogućuje korištenje GPU-a za izračune i minimizira potrebu za prijenosom podataka između CPU-a i GPU-a.

Sada možemo izgraditi naš generator mreže i započeti treniranje. Može se temeljiti na bilo kojoj rekurentnoj ćeliji koju smo raspravili u prethodnoj jedinici (jednostavna, LSTM ili GRU). U našem primjeru koristit ćemo LSTM.

Budući da mreža prima znakove kao ulaz, a veličina vokabulara je prilično mala, ne trebamo sloj za ugrađivanje; ulaz kodiran u one-hot formatu može direktno ući u LSTM ćeliju. Izlazni sloj bit će `Dense` klasifikator koji će pretvoriti LSTM izlaz u brojeve tokena kodirane u one-hot formatu.

Osim toga, budući da radimo s nizovima promjenjive duljine, možemo koristiti sloj `Masking` za stvaranje maske koja će ignorirati popunjeni dio niza. Ovo nije strogo potrebno, jer nas ne zanima previše sve što dolazi nakon `<eos>` tokena, ali ćemo ga koristiti radi stjecanja iskustva s ovom vrstom sloja. `input_shape` će biti `(None, vocab_size)`, gdje `None` označava niz promjenjive duljine, a izlazni oblik također je `(None, vocab_size)`, kao što možete vidjeti iz `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generiranje izlaza

Sada kada smo model istrenirali, želimo ga koristiti za generiranje izlaza. Prije svega, trebamo način za dekodiranje teksta predstavljenog nizom brojeva tokena. Za to bismo mogli koristiti funkciju `tokenizer.sequences_to_texts`; međutim, ona ne funkcionira dobro s tokenizacijom na razini znakova. Stoga ćemo uzeti rječnik tokena iz tokenizatora (nazvan `word_index`), izgraditi obrnuti mapiranje i napisati vlastitu funkciju za dekodiranje:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Sada ćemo započeti generiranje. Počet ćemo s nekim nizom `start`, kodirati ga u sekvencu `inp`, a zatim ćemo u svakom koraku pozvati našu mrežu kako bismo odredili sljedeći znak.

Izlaz mreže `out` je vektor s `vocab_size` elemenata koji predstavljaju vjerojatnosti svakog tokena, a najvjerojatniji broj tokena možemo pronaći koristeći `argmax`. Zatim dodajemo ovaj znak generiranom popisu tokena i nastavljamo s generiranjem. Ovaj proces generiranja jednog znaka ponavlja se `size` puta kako bismo generirali potreban broj znakova, a završavamo ranije ako se naiđe na `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Uzorkovanje rezultata tijekom treniranja

Budući da nemamo korisne metrike poput *točnosti*, jedini način na koji možemo vidjeti da naš model postaje bolji jest **uzorkovanjem** generiranog niza tijekom treniranja. Da bismo to učinili, koristit ćemo **povratne pozive** (callbacks), tj. funkcije koje možemo proslijediti funkciji `fit`, a koje će se periodično pozivati tijekom treniranja.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Ovaj primjer već generira prilično dobar tekst, ali može se dodatno poboljšati na nekoliko načina:

* **Više teksta**. Koristili smo samo naslove za naš zadatak, ali možda biste htjeli eksperimentirati s punim tekstom. Imajte na umu da RNN-ovi nisu previše dobri u rukovanju dugim sekvencama, pa ima smisla ili ih podijeliti na kraće rečenice, ili uvijek trenirati na fiksnoj duljini sekvence neke unaprijed definirane vrijednosti `num_chars` (recimo, 256). Možete pokušati promijeniti gornji primjer u takvu arhitekturu, koristeći [službeni Keras vodič](https://keras.io/examples/generative/lstm_character_level_text_generation/) kao inspiraciju.

* **Višeslojni LSTM**. Ima smisla isprobati 2 ili 3 sloja LSTM ćelija. Kao što smo spomenuli u prethodnoj jedinici, svaki sloj LSTM-a izdvaja određene uzorke iz teksta, a u slučaju generatora na razini znakova možemo očekivati da će niži LSTM sloj biti odgovoran za izdvajanje slogova, a viši slojevi - za riječi i kombinacije riječi. Ovo se jednostavno može implementirati prosljeđivanjem parametra broja slojeva konstruktoru LSTM-a.

* Također biste mogli eksperimentirati s **GRU jedinicama** i vidjeti koje bolje funkcioniraju, kao i s **različitim veličinama skrivenih slojeva**. Prevelik skriveni sloj može rezultirati prekomjernim učenjem (npr. mreža će naučiti točan tekst), dok manja veličina možda neće dati dobar rezultat.


## Generiranje mekog teksta i temperatura

U prethodnoj definiciji funkcije `generate`, uvijek smo birali znak s najvećom vjerojatnošću kao sljedeći znak u generiranom tekstu. To je često rezultiralo time da se tekst "vrtio" između istih sekvenci znakova iznova i iznova, kao u ovom primjeru:
```
today of the second the company and a second the company ...
```

Međutim, ako pogledamo raspodjelu vjerojatnosti za sljedeći znak, može se dogoditi da razlika između nekoliko najvećih vjerojatnosti nije velika, npr. jedan znak može imati vjerojatnost 0.2, dok drugi ima 0.19, itd. Na primjer, kada tražimo sljedeći znak u sekvenci '*play*', sljedeći znak može jednako dobro biti razmak ili **e** (kao u riječi *player*).

To nas dovodi do zaključka da nije uvijek "pravedno" odabrati znak s većom vjerojatnošću, jer odabir drugog po redu također može dovesti do smislenog teksta. Mudrije je **uzorkovati** znakove iz raspodjele vjerojatnosti koju daje izlaz mreže.

Ovo uzorkovanje može se provesti pomoću funkcije `np.multinomial`, koja implementira tzv. **multinomialnu raspodjelu**. Funkcija koja implementira ovo **meko** generiranje teksta definirana je u nastavku:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Uveli smo još jedan parametar nazvan **temperature** koji se koristi za označavanje koliko strogo trebamo slijediti najveću vjerojatnost. Ako je temperatura 1.0, provodimo pošteno multinomijalno uzorkovanje, a kada temperatura ide prema beskonačnosti - sve vjerojatnosti postaju jednake i nasumično biramo sljedeći znak. U primjeru ispod možemo primijetiti da tekst postaje besmislen kada previše povećamo temperaturu, a nalikuje "cikliranom" strogo generiranom tekstu kada se približi 0.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja mogu proizaći iz korištenja ovog prijevoda.
